<div style="background-color: #04D7FD; padding: 20px; text-align: left;">
    <h1 style="color: #000000; font-size: 36px; margin: 0;">Data Processing for RAG with Data Prep Kit (Python)</h1>
    
</div>


## Before Running the notebook

Please complete [setting up python dev environment](./setup-python-dev-env.md)

## Overview

This notebook will process PDF documents as part of RAG pipeline

TODO : update diagram

![](media/rag-overview-2.png)

This notebook will perform steps 1, 2, 3 and 4 in RAG pipeline.

Here are the processing steps:

- **pdf2parquet** : Extract text (in markdown format) from PDF and store them as parquet files
- **Exact Dedup**: Documents with exact content are filtered out
- **Chunk documents**: Split the PDFs into 'meaningful sections' (paragraphs, sentences ..etc)
- **Text encoder**: Convert chunks into vectors using embedding models

## Step-1: Configuration

In [13]:
from my_config import MY_CONFIG

## Step-2:  Data

We will use white papers  about LLMs.  

- [Granite Code Models](https://arxiv.org/abs/2405.04324)
- [Attention is all you need](https://arxiv.org/abs/1706.03762)

You can of course substite your own data below

### 2.1 - Download data

In [14]:
import os, sys
import shutil
from utils import download_file

shutil.rmtree(MY_CONFIG.INPUT_DATA_DIR, ignore_errors=True)
shutil.os.makedirs(MY_CONFIG.INPUT_DATA_DIR, exist_ok=True)
print ("✅ Cleared input directory")
 
download_file (url = 'https://arxiv.org/pdf/1706.03762', local_file = os.path.join(MY_CONFIG.INPUT_DATA_DIR, 'attention.pdf' ))
download_file (url = 'https://arxiv.org/pdf/2405.04324', local_file = os.path.join(MY_CONFIG.INPUT_DATA_DIR, 'granite.pdf' ))
download_file (url = 'https://arxiv.org/pdf/2405.04324', local_file = os.path.join(MY_CONFIG.INPUT_DATA_DIR, 'granite2.pdf' )) # duplicate


✅ Cleared input directory

input/attention.pdf (2.22 MB) downloaded successfully.

input/granite.pdf (1.27 MB) downloaded successfully.

input/granite2.pdf (1.27 MB) downloaded successfully.


### 2.2 - Set input/output path variables for the pipeline

In [15]:
import os, sys
import shutil

if not os.path.exists(MY_CONFIG.INPUT_DATA_DIR ):
    raise Exception (f"❌ Input folder MY_CONFIG.INPUT_DATA_DIR = '{MY_CONFIG.INPUT_DATA_DIR}' not found")

output_parquet_dir = os.path.join (MY_CONFIG.OUTPUT_FOLDER, '01_parquet_out')
output_exact_dedupe_dir = os.path.join (MY_CONFIG.OUTPUT_FOLDER, '02_dedupe_out')
output_chunk_dir = os.path.join (MY_CONFIG.OUTPUT_FOLDER, '03_chunk_out')
output_embeddings_dir = os.path.join (MY_CONFIG.OUTPUT_FOLDER, '04_embeddings_out')

## clear output folder
shutil.rmtree(MY_CONFIG.OUTPUT_FOLDER, ignore_errors=True)
shutil.os.makedirs(MY_CONFIG.OUTPUT_FOLDER, exist_ok=True)

print ("✅ Cleared output directory")

✅ Cleared output directory


## Step-3: pdf2parquet -  Convert data from PDF to Parquet

This step is reading the input folder containing all PDF files and ingest them in a parquet table using the [Docling package](https://github.com/DS4SD/docling).
The documents are converted into a JSON format which allows to easily chunk it in the later steps.



### 3.1 - Execute 

In [16]:
%%time 

from dpk_pdf2parquet.transform_python import Pdf2Parquet

STAGE = 1 
print (f"🏃🏼 STAGE-{STAGE}: Processing input='{MY_CONFIG.INPUT_DATA_DIR}' --> output='{output_parquet_dir}'\n", flush=True)

result = Pdf2Parquet(input_folder= MY_CONFIG.INPUT_DATA_DIR,
               output_folder= output_parquet_dir,
               data_files_to_use=['.pdf'],
               #pdf2parquet_contents_type='application/json'
               pdf2parquet_contents_type='text/markdown'
               ).transform()

if result == 0:
    print (f"✅ Stage:{STAGE} completed successfully")
else:
    raise Exception (f"❌ Stage:{STAGE}  failed")

🏃🏼 STAGE-1: Processing input='input' --> output='output/01_parquet_out'



22:01:12 INFO - pdf2parquet parameters are : {'batch_size': -1, 'artifacts_path': None, 'contents_type': <pdf2parquet_contents_types.MARKDOWN: 'text/markdown'>, 'do_table_structure': True, 'do_ocr': True, 'ocr_engine': <pdf2parquet_ocr_engine.EASYOCR: 'easyocr'>, 'bitmap_area_threshold': 0.05, 'pdf_backend': <pdf2parquet_pdf_backend.DLPARSE_V2: 'dlparse_v2'>, 'double_precision': 8}
22:01:12 INFO - pipeline id pipeline_id
22:01:12 INFO - code location None
22:01:12 INFO - data factory data_ is using local data access: input_folder - input output_folder - output/01_parquet_out
22:01:12 INFO - data factory data_ max_files -1, n_sample -1
22:01:12 INFO - data factory data_ Not using data sets, checkpointing False, max files -1, random samples -1, files to use ['.pdf'], files to checkpoint ['.parquet']
22:01:12 INFO - orchestrator pdf2parquet started at 2025-01-12 22:01:12
22:01:12 INFO - Number of files is 3, source profile {'max_file_size': 2.112621307373047, 'min_file_size': 1.2146415710

Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

ERR#: COULD NOT CONVERT TO RS THIS TABLE TO COMPUTE SPANS


22:01:29 INFO - Completed 1 files (33.33%) in 0.222 min
22:01:58 INFO - Completed 2 files (66.67%) in 0.713 min
22:02:27 INFO - Completed 3 files (100.0%) in 1.205 min
22:02:27 INFO - Done processing 3 files, waiting for flush() completion.
22:02:27 INFO - done flushing in 0.0 sec
22:02:28 INFO - Completed execution in 1.258 min, execution result 0


✅ Stage:1 completed successfully
CPU times: user 3min 16s, sys: 11.2 s, total: 3min 27s
Wall time: 1min 15s


### 3.2 -  Inspect Generated output

Here we should see one entry per input file processed

In [17]:
from utils import read_parquet_files_as_df

output_df = read_parquet_files_as_df(output_parquet_dir)

# print ("Output dimensions (rows x columns)= ", output_df.shape)

output_df.head(5)

## To display certain columns
#parquet_df[['column1', 'column2', 'column3']].head(5)

,filename,contents,num_pages,num_tables,num_doc_elements,document_id,document_hash,ext,hash,size,date_acquired,pdf_convert_time,source_filename
0,attention.pdf,"Provided proper attribution is provided, Googl...",15,6,147,47e22c7b-4fcb-49ce-a8fb-1a25f8d92d34,2949302674760005271,pdf,f1f600333e46c5d7e23f5a110a903ee38aab0bf7047eca...,46040,2025-01-12T22:01:29.018346,13.329675,attention.pdf
1,granite2.pdf,## Granite Code Models: A Family of Open Found...,28,19,295,098802e4-cd13-4724-b334-3aabfd78d410,3127757990743433032,pdf,0534b6a29ab9cedf21c3f6cf756cf0252d19a8e9135a41...,127678,2025-01-12T22:02:27.942004,29.464984,granite2.pdf
2,granite.pdf,## Granite Code Models: A Family of Open Found...,28,19,295,122f1edf-ed60-40c2-83e4-4b3b9ae732ef,3127757990743433032,pdf,0534b6a29ab9cedf21c3f6cf756cf0252d19a8e9135a41...,127678,2025-01-12T22:01:58.425782,29.370334,granite.pdf


## Step-4: Eliminate Duplicate Documents

We have 2 duplicate documnets here : `granite.pdf` and `granite2.pdf`.

Note how the `hash` for these documents are same.

We are going to perform **de-dupe**

On the content of each document, a SHA256 hash is computed, followed by de-duplication of record having identical hashes.

[Dedupe transform documentation](https://github.com/IBM/data-prep-kit/blob/dev/transforms/universal/ededup/README.md)

### 4.1 - Execute 

In [18]:
%%time 

from dpk_ededup.transform_python import Ededup

STAGE = 2
input_df = read_parquet_files_as_df(output_parquet_dir)  ## for debug purposes
print (f"🏃🏼 STAGE-{STAGE}: Processing input='{output_parquet_dir}' --> output='{output_exact_dedupe_dir}'\n", flush=True)

result = Ededup(input_folder=output_parquet_dir,
    output_folder=output_exact_dedupe_dir,
    ededup_doc_column="contents",
    ededup_doc_id_column="hash"
    ).transform()

if result == 0:
    print (f"✅ Stage:{STAGE} completed successfully")
else:
    raise Exception (f"❌ Stage:{STAGE}  failed")

🏃🏼 STAGE-2: Processing input='output/01_parquet_out' --> output='output/02_dedupe_out'



22:02:28 INFO - exact dedup params are {'doc_column': 'contents', 'doc_id_column': 'hash', 'use_snapshot': False, 'snapshot_directory': None}
22:02:28 INFO - pipeline id pipeline_id
22:02:28 INFO - code location None
22:02:28 INFO - data factory data_ is using local data access: input_folder - output/01_parquet_out output_folder - output/02_dedupe_out
22:02:28 INFO - data factory data_ max_files -1, n_sample -1
22:02:28 INFO - data factory data_ Not using data sets, checkpointing False, max files -1, random samples -1, files to use ['.parquet'], files to checkpoint ['.parquet']
22:02:28 INFO - orchestrator ededup started at 2025-01-12 22:02:28
22:02:28 INFO - Number of files is 3, source profile {'max_file_size': 0.04436779022216797, 'min_file_size': 0.02082538604736328, 'total_file_size': 0.10954761505126953}
22:02:28 INFO - Starting from the beginning
22:02:28 INFO - Completed 1 files (33.33%) in 0.0 min
22:02:28 INFO - Completed 2 files (66.67%) in 0.0 min
22:02:28 INFO - Completed 

✅ Stage:2 completed successfully
CPU times: user 45 ms, sys: 6.63 ms, total: 51.6 ms
Wall time: 44.6 ms


### 4.2 - Inspect Generated output

We would see 2 documents: `attention.pdf`  and `granite.pdf`.  The duplicate `granite.pdf` has been filtered out!

In [19]:
from utils import read_parquet_files_as_df

output_df = read_parquet_files_as_df(output_exact_dedupe_dir)

# print ("Input data dimensions (rows x columns)= ", input_df.shape)
# print ("Output data dimensions (rows x columns)= ", output_df.shape)
print (f"Input files before exact dedupe : {input_df.shape[0]:,}")
print (f"Output files after exact dedupe : {output_df.shape[0]:,}")
print ("Duplicate files removed :  ", (input_df.shape[0] - output_df.shape[0]))

output_df.sample(min(3, output_df.shape[0]))

Input files before exact dedupe : 3
Output files after exact dedupe : 2
Duplicate files removed :   1


,filename,contents,num_pages,num_tables,num_doc_elements,document_id,document_hash,ext,hash,size,date_acquired,pdf_convert_time,source_filename,removed
1,granite.pdf,## Granite Code Models: A Family of Open Found...,28,19,295,122f1edf-ed60-40c2-83e4-4b3b9ae732ef,3127757990743433032,pdf,0534b6a29ab9cedf21c3f6cf756cf0252d19a8e9135a41...,127678,2025-01-12T22:01:58.425782,29.370334,granite.pdf,[]
0,attention.pdf,"Provided proper attribution is provided, Googl...",15,6,147,47e22c7b-4fcb-49ce-a8fb-1a25f8d92d34,2949302674760005271,pdf,f1f600333e46c5d7e23f5a110a903ee38aab0bf7047eca...,46040,2025-01-12T22:01:29.018346,13.329675,attention.pdf,[]


##  Step-5: Doc chunks

Split the documents in chunks.

[Chunking transform documentation](https://github.com/IBM/data-prep-kit/blob/dev/transforms/language/doc_chunk/README.md)

**Experiment with chunking size to find the setting that works best for your documents**

### 5.1 - Execute 

In [20]:
%%time

from dpk_doc_chunk.transform_python import DocChunk

STAGE = 3
input_df = read_parquet_files_as_df(output_exact_dedupe_dir)  ## for debug purposes
print (f"🏃🏼 STAGE-{STAGE}: Processing input='{output_exact_dedupe_dir}' --> output='{output_chunk_dir}'\n", flush=True)


result = DocChunk(input_folder=output_exact_dedupe_dir,
        output_folder=output_chunk_dir,
        doc_chunk_chunking_type= "li_markdown",
        # doc_chunk_chunking_type= "dl_json",
        doc_chunk_chunk_size_tokens = 128,  # default 128
        doc_chunk_chunk_overlap_tokens=30   # default 30
        ).transform()

if result == 0:
    print (f"✅ Stage:{STAGE} completed successfully")
else:
    raise Exception (f"❌ Stage:{STAGE}  failed")

🏃🏼 STAGE-3: Processing input='output/02_dedupe_out' --> output='output/03_chunk_out'



22:02:28 INFO - doc_chunk parameters are : {'chunking_type': 'li_markdown', 'content_column_name': 'contents', 'doc_id_column_name': 'document_id', 'output_chunk_column_name': 'contents', 'output_source_doc_id_column_name': 'source_document_id', 'output_jsonpath_column_name': 'doc_jsonpath', 'output_pageno_column_name': 'page_number', 'output_bbox_column_name': 'bbox', 'chunk_size_tokens': 128, 'chunk_overlap_tokens': 30, 'dl_min_chunk_len': None}
22:02:28 INFO - pipeline id pipeline_id
22:02:28 INFO - code location None
22:02:28 INFO - data factory data_ is using local data access: input_folder - output/02_dedupe_out output_folder - output/03_chunk_out
22:02:28 INFO - data factory data_ max_files -1, n_sample -1
22:02:28 INFO - data factory data_ Not using data sets, checkpointing False, max files -1, random samples -1, files to use ['.parquet'], files to checkpoint ['.parquet']
22:02:28 INFO - orchestrator doc_chunk started at 2025-01-12 22:02:28
22:02:28 INFO - Number of files is 3,

✅ Stage:3 completed successfully
CPU times: user 55.3 ms, sys: 1.12 ms, total: 56.4 ms
Wall time: 49.4 ms


### 5.2 - Inspect Generated output

We would see documents are split into many chunks

In [21]:
from utils import read_parquet_files_as_df

output_df = read_parquet_files_as_df(output_chunk_dir)

print (f"Files processed : {input_df.shape[0]:,}")
print (f"Chunks created : {output_df.shape[0]:,}")

# print ("Input data dimensions (rows x columns)= ", input_df.shape)
# print ("Output data dimensions (rows x columns)= ", output_df.shape)

output_df.sample(min(3, output_df.shape[0]))

Files processed : 2
Chunks created : 60


,filename,num_pages,num_tables,num_doc_elements,document_hash,ext,hash,size,date_acquired,pdf_convert_time,source_filename,removed,source_document_id,contents,document_id
39,granite.pdf,28,19,295,3127757990743433032,pdf,0534b6a29ab9cedf21c3f6cf756cf0252d19a8e9135a41...,127678,2025-01-12T22:01:58.425782,29.370334,granite.pdf,[],122f1edf-ed60-40c2-83e4-4b3b9ae732ef,## 4.3 Optimization\n\nWe use AdamW optimizer ...,c45f847235c42841ae094834f1eb950f774e314f46566e...
53,granite.pdf,28,19,295,3127757990743433032,pdf,0534b6a29ab9cedf21c3f6cf756cf0252d19a8e9135a41...,127678,2025-01-12T22:01:58.425782,29.370334,granite.pdf,[],122f1edf-ed60-40c2-83e4-4b3b9ae732ef,## 6.5 Math Reasoning\n\nWe use the following ...,cc28f6c0c14fda7a2f3c36b0874a5082e20c6e4e57d0f3...
29,granite.pdf,28,19,295,3127757990743433032,pdf,0534b6a29ab9cedf21c3f6cf756cf0252d19a8e9135a41...,127678,2025-01-12T22:01:58.425782,29.370334,granite.pdf,[],122f1edf-ed60-40c2-83e4-4b3b9ae732ef,## 1 Introduction\n\nOver the last several dec...,716c16eb39724cce53c0fffc485b7ef8e3963534087fc3...


## Step-6:   Calculate Embeddings for Chunks

we will calculate embeddings for each chunk using an open source embedding model

[Embeddings / Text Encoder documentation](https://github.com/IBM/data-prep-kit/blob/dev/transforms/language/text_encoder/README.md)

### 6.1 - Execute

In [22]:
%%time 

from dpk_text_encoder.transform_python import TextEncoder

STAGE  = 4
input_df = read_parquet_files_as_df(output_chunk_dir)  ## for debug purposes
print (f"🏃🏼 STAGE-{STAGE}: Processing input='{output_chunk_dir}' --> output='{output_embeddings_dir}'\n", flush=True)


result = TextEncoder(input_folder= output_chunk_dir, 
               output_folder= output_embeddings_dir, 
               text_encoder_model_name = MY_CONFIG.EMBEDDING_MODEL
               ).transform()
if result == 0:
    print (f"✅ Stage:{STAGE} completed successfully")
else:
    raise Exception (f"❌ Stage:{STAGE}  failed")

🏃🏼 STAGE-4: Processing input='output/03_chunk_out' --> output='output/04_embeddings_out'



22:02:28 INFO - text_encoder parameters are : {'content_column_name': 'contents', 'output_embeddings_column_name': 'embeddings', 'model_name': 'sentence-transformers/all-MiniLM-L6-v2'}
22:02:28 INFO - pipeline id pipeline_id
22:02:28 INFO - code location None
22:02:28 INFO - data factory data_ is using local data access: input_folder - output/03_chunk_out output_folder - output/04_embeddings_out
22:02:28 INFO - data factory data_ max_files -1, n_sample -1
22:02:28 INFO - data factory data_ Not using data sets, checkpointing False, max files -1, random samples -1, files to use ['.parquet'], files to checkpoint ['.parquet']
22:02:28 INFO - orchestrator text_encoder started at 2025-01-12 22:02:28
22:02:28 INFO - Number of files is 2, source profile {'max_file_size': 0.04669189453125, 'min_file_size': 0.02893352508544922, 'total_file_size': 0.07562541961669922}
22:02:30 INFO - Completed 1 files (50.0%) in 0.002 min
22:02:30 INFO - Completed 2 files (100.0%) in 0.006 min
22:02:30 INFO - Don

✅ Stage:4 completed successfully
CPU times: user 540 ms, sys: 50.4 ms, total: 591 ms
Wall time: 2.54 s


### 6.2 - Inspect Generated output

In [23]:
from utils import read_parquet_files_as_df

output_df = read_parquet_files_as_df(output_embeddings_dir)

print ("Input data dimensions (rows x columns)= ", input_df.shape)
print ("Output data dimensions (rows x columns)= ", output_df.shape)

output_df.sample(min(3, output_df.shape[0]))

Input data dimensions (rows x columns)=  (60, 15)
Output data dimensions (rows x columns)=  (60, 16)


,filename,num_pages,num_tables,num_doc_elements,document_hash,ext,hash,size,date_acquired,pdf_convert_time,source_filename,removed,source_document_id,contents,document_id,embeddings
4,attention.pdf,15,6,147,2949302674760005271,pdf,f1f600333e46c5d7e23f5a110a903ee38aab0bf7047eca...,46040,2025-01-12T22:01:29.018346,13.329675,attention.pdf,[],47e22c7b-4fcb-49ce-a8fb-1a25f8d92d34,## 2 Background\n\nThe goal of reducing sequen...,8092df4b65063ad84b121780ba3570ef985a2b6bd04782...,"[-0.09019503, -0.03779276, 0.0051401607, -0.04..."
34,granite.pdf,28,19,295,3127757990743433032,pdf,0534b6a29ab9cedf21c3f6cf756cf0252d19a8e9135a41...,127678,2025-01-12T22:01:58.425782,29.370334,granite.pdf,[],122f1edf-ed60-40c2-83e4-4b3b9ae732ef,## 2.4 Natural Language Datasets\n\nIn additio...,1289d564d558c095b4d4ba62c845f33bf6c666ea8d9a13...,"[-0.05261178, -0.082296796, 0.03544874, 0.0172..."
3,attention.pdf,15,6,147,2949302674760005271,pdf,f1f600333e46c5d7e23f5a110a903ee38aab0bf7047eca...,46040,2025-01-12T22:01:29.018346,13.329675,attention.pdf,[],47e22c7b-4fcb-49ce-a8fb-1a25f8d92d34,## 1 Introduction\n\nRecurrent neural networks...,2ea59b12da7c45067147586b86dc913164b68c7da14680...,"[-0.04227975, -0.0830809, 0.02563228, 0.018819..."


## Step-7: Copy output to final output dir

In [24]:
import shutil

shutil.rmtree(MY_CONFIG.OUTPUT_FOLDER_FINAL, ignore_errors=True)
shutil.copytree(src=output_embeddings_dir, dst=MY_CONFIG.OUTPUT_FOLDER_FINAL)

print (f"✅ Copied output from '{output_embeddings_dir}' --> '{MY_CONFIG.OUTPUT_FOLDER_FINAL}'")

✅ Copied output from 'output/04_embeddings_out' --> 'output/output_final'
